## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions


In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
model

/Users/yashsarjekar/Documents/AI_WORK/Langchain/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x16829de80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x16829eba0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Nice to meet you, Krish. It's great to have a Chief AI Engineer like you on board. As a Chief AI Engineer, I'm sure you've been at the forefront of AI innovation and have valuable insights to share. What areas of AI are you most interested in, and what are some of the exciting projects you're currently working on?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 48, 'total_tokens': 119, 'completion_time': 0.114861744, 'completion_tokens_details': None, 'prompt_time': 0.002343529, 'prompt_tokens_details': None, 'queue_time': 0.054837661, 'total_time': 0.117205273}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b6182-79cd-7732-8bdc-17153288609f-0', usage_metadata={'input_tokens': 48, 'output_tokens': 71, 'total_tokens': 119})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="Your name is Krish, and you're a Chief AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 117, 'total_tokens': 131, 'completion_time': 0.016205702, 'completion_tokens_details': None, 'prompt_time': 0.008097772, 'prompt_tokens_details': None, 'queue_time': 0.050167038, 'total_time': 0.024303474}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b6184-395a-7482-b16f-56da8076f5b5-0', usage_metadata={'input_tokens': 117, 'output_tokens': 14, 'total_tokens': 131})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [8]:
response.content

'Hello Krish, nice to meet you. As a Chief AI Engineer, you must be working on some exciting projects that involve the development and implementation of artificial intelligence and machine learning models. What specific areas of AI are you currently focusing on, and what challenges are you trying to solve?'

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content="Your name is Krish, and you're a Chief AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 119, 'total_tokens': 133, 'completion_time': 0.016262355, 'completion_tokens_details': None, 'prompt_time': 0.006909665, 'prompt_tokens_details': None, 'queue_time': 0.055319535, 'total_time': 0.02317202}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b618e-c7dc-70a1-9bbf-a8fd6c0aa20c-0', usage_metadata={'input_tokens': 119, 'output_tokens': 14, 'total_tokens': 133})

In [10]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"I don't have any information about your name as our conversation just started, and I don't retain any data from previous conversations. If you'd like to share your name with me, I can use it in our conversation."

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [11]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [12]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [13]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Nice to meet you, Krish! I'm happy to assist you with any questions or topics you'd like to discuss. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 57, 'total_tokens': 90, 'completion_time': 0.041934387, 'completion_tokens_details': None, 'prompt_time': 0.002856791, 'prompt_tokens_details': None, 'queue_time': 0.055152559, 'total_time': 0.044791178}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b6195-a43c-79e2-910b-c53c1e18fbd2-0', usage_metadata={'input_tokens': 57, 'output_tokens': 33, 'total_tokens': 90})

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish!'

In [15]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [16]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [17]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते Krish! मैं आपकी मदद करने के लिए यहाँ हूँ। क्या आपके पास कोई प्रश्न या जानकारी चाहिए?'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages